Testing an Automation Listener

In [ ]:
@Operation(id = "Scripting.Listener")
function run(input, params) {

    Console.log("from js listener");
    // check input
    Console.logAsJson(input);
    Console.logAsJson(params);

    // dump the whole context
    Debug.dumpCtx();
    
    // introspection example
    Console.log("Introspection Example");    
    Debug.introspect(ctx.Event);
    
}

Do a test running the script from inside a synchronous listener

In [ ]:
Harness.registerListener('documentCreated','Scripting.Listener',true);


var root = Repository.GetDocument(null, {
        "value" : "/"
    });

var newDoc = Document.Create(root, {
        "type" : "File",
        "name" : "newDoc",
        "properties" : {
            "dc:title" : "whathever"
        }
    });

    

Do a test running the script from inside an  **a**synchronous listener

In [ ]:
Console.log("starting listener testing");

Harness.registerListener('documentCreated','Scripting.Listener',false);

var root = Repository.GetDocument(null, {
        "value" : "/"
    });

var newDoc = Document.Create(root, {
        "type" : "File",
        "name" : "newDoc",
        "properties" : {
            "dc:title" : "whathever"
        }
    });

Console.log("created test Document " + newDoc.id);

Harness.waitForAsync();

Console.log("end of listener testing");